# Multi-Objective Ackley Function
# optimized using MOO Easy Tool

## Initialization

### MOO imports

In [5]:
import requests
import os
## Set API back-end direction
backend="http://138.100.20.80/api/"

### Create new experiment

In [6]:
experiment = {
    "name": "MOAckley_1inputd",
    "n_ins": 1, 
    "input_names": ["x"], 
    "input_mms": [[-2, 2]],
    "n_objs": 2,
    "objective_names": ["f1(x)","f2(x)"],
    "objective_mms": [False, False],
    "n_cons": 0,
    "kernel": 'RBF',
    "acq_id": 2,
    "acqfunct_hps": {"N": 1000, "M": 50}
}

response = requests.post(backend+'startexp', json=experiment)
print(response.__dict__)
if response.status_code==200:
    response = response.json()
    for key in response:
        experiment[key]=response[key]

print(experiment)

{'_content': b'{"id":60,"name":"MOAckley_1inputd","n_ins":1,"input_names":["x"],"input_mms":[[-2.0,2.0]],"n_objs":2,"objective_names":["f1(x)","f2(x)"],"objective_mms":[false,false],"n_cons":0,"kernel":null,"acq_id":2,"acqfunct_hps":{"N":1000,"M":50},"X":null,"Y":null,"next_x":null,"adh":null,"agd":null,"ns":null}', '_content_consumed': True, '_next': None, 'status_code': 200, 'headers': {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Mon, 25 Apr 2022 10:22:46 GMT', 'Content-Type': 'application/json', 'Content-Length': '300', 'Connection': 'keep-alive'}, 'raw': <urllib3.response.HTTPResponse object at 0x7fba01360b70>, 'url': 'http://138.100.20.80/api/startexp', 'encoding': 'utf-8', 'history': [], 'reason': 'OK', 'cookies': <RequestsCookieJar[]>, 'elapsed': datetime.timedelta(microseconds=458282), 'request': <PreparedRequest [POST]>, 'connection': <requests.adapters.HTTPAdapter object at 0x7fba387d97b8>}
{'name': 'MOAckley_1inputd', 'n_ins': 1, 'input_names': ['x'], 'input_mms': [[-2.0, 2.

### Define Evaluation Function

In [7]:
import numpy as np
def ackley(xx, a=20, b=0.2, c=2*np.pi):

    d = len(xx)

    sum1 = np.sum(xx**2)
    sum2 = np.sum(np.cos(c*xx))

    term1 = -a * np.exp(-b*np.sqrt(sum1/d))
    term2 = -np.exp(sum2/d)

    return term1 + term2 + a + np.exp(1)

def MOOackley(xx, c1=0.5, c2=-0.5, a=20, b=0.2, c=2*np.pi):
    return np.array([ackley(np.array([xx-c1])),ackley(np.array([xx-c2]))])

def evaluation(x):
    return MOOackley(x, c1=-.6, c2=.6)

## First iteration

In [8]:
x_best = np.random.uniform(experiment["input_mms"][0][0], experiment["input_mms"][0][1], experiment["n_ins"])
y_best = evaluation(x_best)
response = requests.post(backend+"setsample/"+str(experiment["id"]), json={"x": list(x_best), "y": list(y_best)})

## Main Loop

In [9]:
for i in range(49):
    response = requests.get(backend+"getsample/next/"+str(experiment["id"]))

    ## Charge next configuration 
    x_best = np.array(response.json()["next_x"])
    y_best = evaluation(x_best)

    response = requests.post(backend+"setsample/"+str(experiment["id"]), json={"x": list(x_best), "y": list(y_best)})